In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.optimize import curve_fit
import pypolar.fresnel as fresnel
from scipy.signal import find_peaks, peak_prominences
%config InlineBackend.figure_format = 'retina'

In [ ]:
file_name = 'data/09-14-2024/refl-air-09-14-2024-01.csv'                                   # Input file name
df = pd.read_csv(file_name)
# Plot curve
df.plot(x="Stage", y="Intensity", alpha=0.5, style='.-')                                      # Plot
plt.show()

In [ ]:
# Find EPI position
# 1. Clip the midle part :
x = df['Intensity'].to_numpy()
peaks_pos, _ = find_peaks(x, prominence=0.5)

# Check prominences if this doesn't work :
# prominences = peak_prominences(x, peaks_pos)[0]
# print(prominences)

epi_peak = peaks_pos[(df['Stage'][peaks_pos[:]] >= 4) & (df['Stage'][peaks_pos[:]] <= 6)]
df.plot(x="Stage", y="Intensity", alpha=0.5, style='.-')
plt.plot(df['Stage'][epi_peak[:]], df['Intensity'][epi_peak[:]], "x")
plt.title('EPI position')
print(df.iloc[epi_peak[:]])

In [ ]:
# Step 2 : Crop the original curve from one end to epi
df_crop = df.iloc[2:epi_peak[0]+1]
# Plot cropped curve
# df_crop.plot(x="Stage", y="Intensity", alpha=0.5, style='.-')                                       # Plot
# plt.show()

# Normalize array 
arr = df_crop['Intensity'].to_numpy()
normalized_arr = (arr - arr.min()) / (arr.max() - arr.min())
plt.plot(df_crop['Stage'],np.flip(normalized_arr), '.-', alpha=0.5)

In [6]:
# Step 3 : Plot fresnel relation for irradiance.
n1 = 1.51   # refractive index of glass
n2 = 1.0    # refractive index of air

In [7]:
stage_pos = df_crop['Stage'].to_numpy()
stage_interp = np.linspace(stage_pos.min(),stage_pos.max(),200)
y_data = np.flip(normalized_arr)
y_interp = np.interp(stage_interp, stage_pos, y_data)
# plt.plot(stage_interp, y_interp, '.-', label='Data')


In [ ]:
# Find minima
min_index = np.argmin(y_interp)
x_min = stage_interp[min_index]


print("Index of global minimum:", min_index)
print("X value at global minimum:", x_min)
print("Y value at global minimum:", y_interp[min_index])

# brewsters_angle = fresnel.brewster(1.515, n_i=1.00028460, deg=True)
brewsters_angle = fresnel.brewster(1.00028460, n_i=1.515, deg=True)

print("Brewster's angle for this interface (deg): ", brewsters_angle)
print("Critical angle for this interface (deg): ", fresnel.critical(n2, n_i=n1, deg=True))

In [ ]:
new_x_half,step_size = np.linspace(0.0,brewsters_angle,min_index, retstep=True)
plt.plot(new_x_half, y_interp[0:111], '.-', label='Data')

In [ ]:
print(len(y_interp)-min_index)
ini_pt = brewsters_angle+step_size
end_pt = 85.0
new_x_other_half = np.arange(ini_pt, end_pt, step_size)
print(len(new_x_other_half[0:len(y_interp)-min_index]))

# Concatenate the two arrays
new_x_half
new_x_other_half[0:len(y_interp)-min_index]

new_x = np.concatenate((new_x_half, new_x_other_half[0:len(y_interp)-min_index]))

In [ ]:
plt.plot(new_x, y_interp, '.-', label='Data')

In [ ]:
def func(x, a,b):
    ang = x+b
    return a*fresnel.R_par(n2, ang, n_i=n1, deg=True)

popt, pcov = curve_fit(func, new_x[0:120], y_interp[0:120])
print(popt)

In [ ]:
y_fresnel = fresnel.R_par(n2, new_x, n_i=n1, deg=True)

plt.plot(new_x, y_interp, '.-', label='Data')
plt.plot(new_x+0.19520471, func(new_x,*popt), '.-', label='Fit')
plt.plot(new_x, y_fresnel, 'r.-', label='Fresnel')
plt.ylim([0,1.0])
plt.legend()

In [ ]:
calibration_angles = new_x+0.19520471
stage_interp
plt.plot(stage_interp, np.flip(calibration_angles), '.-', label='Angles')
plt.title('Incident angle vs position')

In [ ]:
data = np.column_stack((stage_interp, np.flip(calibration_angles), y_interp, y_fresnel))
df2 = pd.DataFrame({'Stage': data[:, 0], 'Angle': data[:, 1], 'Data': data[:, 2], 'Fresnel': data[:, 3]})
# file_name = 'data/09-14-2024/air-calibration.csv'
# df2.to_csv(file_name, index=False)